In [12]:
import csv
''' Depois de tratar o texto, e guardar os csv's em (letras.csv), (nutricao.csv)
    Podemos de fato começar a aplicar o método do simplex para determinar as melhores dietas'''

nomes = ['HENRIQUE','LUCAS','RAFAEL'] #nossos nomes

#nutrientes da que nossa dieta precisa satisfazer 
nutrientes_minimos = ['Caloria','Calcio','VitaminaA','Riboflavina','AcidoAscorbico'] 

dieta_prodx_por_letra = dict() #dicionario de letra para dieta e prodx
with open('letras.csv',newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        dieta_prodx_por_letra[row['Letra']] = (float(row['Dieta']),float(row['ProdutoX']))

Vamos definir agora nosso problema de minimização!

Se gostaríamos de reduzir o custo, nossas variáveis serão todas $x_i$, onde $x_i$ significa quantos dólares gastamos no alimento $i$.
Como o médico nos passou quantidades mínimas para suprir de nutrientes, essas quantidades formarão nossa matriz de restrição.

Teremos algo do tipo:
$$
    x_1c_1 + x_2c_2 + ... + x_nc_n \geq c\\
    x_1p_1 + x_2p_2 + ... + x_np_n \geq p\\
    x_1v_1 + x_2v_2 + ... + x_nv_n \geq v\\
    x_1r_1 + x_2r_2 + ... + x_nr_n \geq r\\
    x_1a_1 + x_2a_2 + ... + x_na_n \geq a\\
$$

Onde $c_i$ é quanto de calórias o alimento $i$ dá por unidade de dolar gasto e $c$ é a quantidade mínima de calorias que o médico nos recomendou. (Fiz a mesma coisa para os outros nutrientes)

Como isso não está exatamente na forma padrão, teremos que multiplicar por -1 todas as linhas, de forma a tornar todo $\geq$ em $\leq$.

In [13]:
''' ideia 
M = [ [c1,c2,c3,...,cn],
      [p1,p2,p3,...,pn],
      ...
    ]
'''
alimentos = []
with open('nutricao.csv','r',newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        alimentos.append(row)

In [14]:
mercadorias_disponiveis = ['Farinhadetrigo(enriquecida)','Leite evaporado (lata)','Queijo (cheddar)',
                     'Fígado (boi)','Espinafre','Batata-doce','Feijão verde. seco']

alimentos_disponiveis = []
for a in alimentos:
    if a['Mercadoria'] in mercadorias_disponiveis:
        alimentos_disponiveis.append(a)
alimentos_disponiveis

[{'Numero': '1',
  'Mercadoria': 'Farinhadetrigo(enriquecida)',
  'Caloria': '44.7',
  'Proteina': '1411',
  'Calcio': '2.0',
  'Ferro': '365',
  'VitaminaA': '0',
  'Tiamina': '55.4',
  'Riboflavina': '33.3',
  'Niacina': '441',
  'AcidoAscorbico': '0'},
 {'Numero': '15',
  'Mercadoria': 'Leite evaporado (lata)',
  'Caloria': '8.4',
  'Proteina': '422',
  'Calcio': '15.1',
  'Ferro': '9',
  'VitaminaA': '26.0',
  'Tiamina': '3.0',
  'Riboflavina': '23.5',
  'Niacina': '11',
  'AcidoAscorbico': '60'},
 {'Numero': '19',
  'Mercadoria': 'Queijo (cheddar)',
  'Caloria': '7.4',
  'Proteina': '448',
  'Calcio': '16.4',
  'Ferro': '19',
  'VitaminaA': '28.1',
  'Tiamina': '0.8',
  'Riboflavina': '10.3',
  'Niacina': '4',
  'AcidoAscorbico': '0'},
 {'Numero': '30',
  'Mercadoria': 'Fígado (boi)',
  'Caloria': '2.2',
  'Proteina': '333',
  'Calcio': '0.2',
  'Ferro': '139',
  'VitaminaA': '169.2',
  'Tiamina': '6.4',
  'Riboflavina': '50.8',
  'Niacina': '316',
  'AcidoAscorbico': '525'},
 {'N

Depois de conseguir resultados estranhos, percebi que as dimensões passadas pelo médico e pelas tabelas estão em dimensões diferentes:
- Dimensão Dieta: **$10^2$ calorias, $10^{-2}$ gramas, $10^2$ unidades internacionais, $10^{-1}$ miligramas e miligrama.**
- Dimensões Produto X: **$10^3$ calorias/dólar, $10^{−1}$ gramas/dólar, $10^3$ unidades internacionais/dólar, $10^{−1}$ miligramas/dólar, miligramas/dólar.**

In [15]:
tabela_sem_prod_x = []
for nutriente in nutrientes_minimos:
    linha_nutri = [float(x[nutriente]) for x in alimentos]
    tabela_sem_prod_x.append(linha_nutri)


In [16]:
tabela_loja = []
for nutriente in nutrientes_minimos:
    linha_nutri = [float(x[nutriente]) for x in alimentos_disponiveis]
    tabela_loja.append(linha_nutri)
tabela_loja

[[44.7, 8.4, 7.4, 2.2, 1.1, 9.6, 17.4],
 [2.0, 15.1, 16.4, 0.2, 0.0, 2.7, 3.7],
 [0.0, 26.0, 28.1, 169.2, 918.4, 290.7, 5.1],
 [33.3, 23.5, 10.3, 50.8, 13.8, 5.4, 38.2],
 [0.0, 60.0, 0.0, 525.0, 2755.0, 1912.0, 0.0]]

In [19]:
#com a matriz de restrição sem o produto X pronta, já estamos quase finalizados, basta descobrir quanto produto X nos dá e o B para cada nome!
from copy import deepcopy
import numpy as np
import scipy.optimize

conversao_dieta = [10**-1, 10**-2, 10**-1, 10**-1, 1] # para conversão da restrição
conversao_prod_x = [1,10**-1,1,10**-1,1] # para conversao do produto x

def matriz_todos_alimentos(nome:str):
    nome = nome.upper()
    m = deepcopy(tabela_sem_prod_x)
    for i in range(5): #são cinco nutrientes
        if i < len(nome) and nome[i] in dieta_prodx_por_letra:
            letra = nome[i]
            val = float(dieta_prodx_por_letra[letra][1]) * conversao_prod_x[i]
            m[i].append(val)
        else: 
            m[i].append(0.)
    return np.matrix(m)

def matriz_loja(nome:str):
    nome = nome.upper()
    m = deepcopy(tabela_loja)
    for i in range(5): #são cinco nutrientes
        if i < len(nome) and nome[i] in dieta_prodx_por_letra:
            letra = nome[i]
            val = float(dieta_prodx_por_letra[letra][1]) * conversao_prod_x[i]
            m[i].append(val)
        else: 
            m[i].append(0.)
    return np.matrix(m)



def restricoes(nome:str):
    nome = nome.upper()
    r = []
    for i in range(5):
        if i < len(nome) and nome[i] in dieta_prodx_por_letra:
            letra = nome[i]
            val = float(dieta_prodx_por_letra[letra][0]) * conversao_dieta[i]
            r.append(val)
        else:
            r.append(0.)
    return np.array(r)



def dieta(nome:str,method:str):
    m = -matriz_loja(nome)
    r = -restricoes(nome)
    c = np.ones(len(alimentos_disponiveis) + 1) # esse é o funcional linear que será minizado
    return scipy.optimize.linprog(c=c,A_ub=m,b_ub=r,method=method)

In [24]:
ar = dieta('henrique','highs').x
ar.shape = (8,1)

matriz_loja('henrique')*ar

matrix([[ 3.        ],
        [ 0.39      ],
        [24.27644967],
        [ 4.7       ],
        [65.        ]])

In [25]:
restricoes('henrique')

array([ 3.  ,  0.39,  0.6 ,  4.7 , 65.  ])

In [21]:
dieta('rafael','highs').x

array([0.00315594, 0.        , 0.        , 0.        , 0.01213675,
       0.        , 0.        , 0.06784447])